# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# Connection to a Cassandra instance to local machine 
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# Session to establish connection and begin executing queries
session = cluster.connect()

#### Create Keyspace

In [6]:
# Creating a Keyspace 
try:
    session.execute("""
                    CREATE KEYSPACE IF NOT EXISTS sparkify
                    WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor':1}""")
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Setting KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

### Create queries to ask the following three questions of the data

###### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and   itemInSession  = 4


###### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10,   sessionid = 182
    

###### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




#### QUERY 1: 
##### Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

Since the query is looking for the data during `sessionId` and `itemInSession`, data ia equally distributed  by using `sessionId`.
Now here`sessionId` is the ***partition key*** and `itemInSession` will be the ***clustering column***

In [8]:
## Query 1
create_songplays = "CREATE TABLE IF NOT EXISTS songplays_info (session_id INT, \
                                    item_in_session SMALLINT,\
                                    artist TEXT,\
                                    song_length DECIMAL,\
                                    song_title TEXT, \
                                    PRIMARY KEY(session_id,item_in_session));"
try:
    session.execute(create_songplays)
except Exception as e:
    print(e)
                    

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songplays_info (session_id,item_in_session,artist,song_length,song_title)"
        query = query + " VALUES(%s,%s,%s,%s,%s)"
        try:
            session.execute(query, (int(line[8]), int(line[3]),line[0],float(line[5]),line[9]))
        except Exception as e:
            print("[0] :{}, [3] : {},[5] : {},[8] : {},[9] : {}".format(line[0], line[3],line[5],line[8],line[9]))
            print(e)

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
## SELECT statement to verify the data was entered into the table
select_query = "SELECT artist,song_title,song_length FROM songplays_info \
                        WHERE session_id = 338 AND item_in_session = 4"
try:
    records = session.execute(select_query)
except Exception as e:
    print(e)

df = pd.DataFrame(list(records))
df

,artist,song_title,song_length
0,Faithless,Music Matters (Mark Knight Dub),495.3073


#### QUERY 2: 
##### Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

Since the query is looking for the data with `userId` and `sessionId`,also data should be sorted based on the `itemInSession`, both `userId` and `sessionId` together form the ***partition key***.
`itemInSession` will be the ***clustering column*** which store the data in ascending order within th epartition for faster retrieval of similar values.

In [11]:
## Query 2: 
create_user_songplays = "CREATE TABLE IF NOT EXISTS user_songplays_info( \
                            user_id INT , \
                            session_id INT, \
                            item_in_session SMALLINT, \
                            artist TEXT ,\
                            first_name TEXT,\
                            last_name TEXT,\
                            song_title TEXT, \
                            PRIMARY KEY((user_id,session_id),item_in_session))"
try:
    session.execute(create_user_songplays)
except Exception as e:
    print(e)                    

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Assigning the INSERT statements into the `query` variable
        query = "INSERT INTO user_songplays_info (user_id, session_id, \
                            item_in_session, artist, first_name, last_name,  song_title )"
        query = query + " VALUES(%s,%s,%s,%s,%s,%s,%s)"
        artist = line[0]
        first_name = line[1]
        item_in_session = int(line[3])
        last_name = line[4]
        session_id = int(line[8])
        song = line[9]
        user_id = int(line[10])
        try:
            session.execute(query, (user_id, session_id, item_in_session, artist, first_name, last_name, song))
        except Exception as e:
            print(line)
            print(e)
            break


In [13]:
## SELECT statement to verify the data was entered into the table
select_query = "SELECT artist,song_title,first_name,last_name FROM user_songplays_info \
                    WHERE user_id = 10 AND session_id = 182 ORDER BY item_in_session"
try:
    result = session.execute(select_query)
except Exception as e:
    print(e)
df = pd.DataFrame(list(result))
df

,artist,song_title,first_name,last_name
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


#### QUERY 3: 
##### Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

Since the query is looking for the data with `song`, data is didtributed evenly based on the `song`.

In [14]:
## Query 3: 
create_user_details = "CREATE TABLE IF NOT EXISTS user_details ( \
                                    song TEXT, \
                                    user_id INT, \
                                    first_name TEXT, \
                                    last_name TEXT, \
                                    PRIMARY KEY (song,user_id))"
try:
    session.execute(create_user_details)
except Exception as e:
    print(e)
                    

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Assigning the INSERT statements into the `query` variable
        query = "INSERT INTO user_details (song, user_id, first_name, last_name) "
        query = query + "VALUES (%s,%s,%s,%s)"
        user_id = int(line[10])
        first_name = line[1]
        last_name = line[4]
        song_title = line[9]
    
        try:
            session.execute(query, (song_title,user_id,first_name,last_name))
        except Exception as e:
            print(e)
            break
#print(line)

In [16]:
## SELECT statement to verify the data was entered into the table

select_query = "SELECT first_name, last_name \
                    FROM user_details \
                    WHERE song='All Hands Against His Own'"
try:
    result3 = session.execute(select_query)
except Exception as e:
    print(e)

df = pd.DataFrame(list(result3))
df
    

,first_name,last_name
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


In [17]:
#Another way to assing column names to the result
#def pandas_factory(colnames, rows):
#    return pd.DataFrame(rows, columns=colnames)

#session.row_factory = pandas_factory

#df = result3._current_rows 
#df

### Drop the tables before closing out the sessions

In [18]:
## Dropping the table before closing out the sessions
drop_songplays = "DROP TABLE songplays_info"
drop_user_songplays = "DROP TABLE user_songplays_info"
drop_user_details = "DROP TABLE user_details"

drop_statements = [drop_songplays,drop_user_songplays,drop_user_details]
for query in drop_statements:
    try:
        session.execute(query)
    except Exception as e:
        print(e)

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()